In [70]:
import pandas as pd
import json
from datetime import datetime, timedelta
import random

In [71]:
csv_file_path = 'TabelaNutrIA-Tratada.csv'
df = pd.read_csv(csv_file_path, sep=',')
jsonl_file_path = 'output-backend.jsonl'

In [72]:
dates = {
    'hoje': datetime.now().strftime('%Y-%m-%d'),
    'ontem': (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'),
    'amanha': (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
}

datas_chaves = list(dates.keys())


In [73]:
content = f"Você é um assistente nutricional especializado e sua função é fornecer respostas claras e diretas sobre tópicos de nutrição, incluindo planejamento de refeições, controle de calorias e dicas para uma alimentação saudável baseados nos pilares de dietas flexível, contagem de macronutrientes e balanço energético. Você também deve retornar informação nutricional sobre alimentos baseado em nossa tabela de alimentos e ajuda os usuários a registrar seu consumo calórico durante o dia em refeições específicas quando solicitado. Você deve gerar em um formato JSON específico para que o sistema de backend possa processar e executar ações conforme necessário. Hoje é dia {dates['hoje']}."

### Resposta Completa

In [74]:
with open(jsonl_file_path, "w", encoding="utf-8") as jsonl_file:
    for index, row in df.iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        qtde_utilizada_porcao = random.randint(1, 5)
        qtde_utilizada_gramas = qtde_utilizada_porcao * row["porcao_padrao"]
        qtde_utilizada = round(qtde_utilizada_gramas / row["porcao_padrao"], 1)
        entrada_usuario = f"Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {row['estado_alimento']} na minha refeicao {numero_refeicao} de {data_chave}."

        arguments = {
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMAS",
                    "porcao_padrao":round(row["porcao_padrao"], 1),
                    "qtde_utilizada":qtde_utilizada,
                    "qtde_proteina":round(qtde_utilizada * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        qtde_utilizada * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(qtde_utilizada * row["qtde_gordura"], 1),
                    "qtde_alcool":round(qtde_utilizada * row["qtde_alcool"], 1),
                    "kcal":round(qtde_utilizada * row["kcal"], 1),
                }

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {"role": "user", "content": entrada_usuario},
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem refeição

In [75]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(50).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        qtde_utilizada_porcao = random.randint(1, 5)
        qtde_utilizada_gramas = qtde_utilizada_porcao * row['porcao_padrao']
        qtde_utilizada = round(qtde_utilizada_gramas/row['porcao_padrao'],1)
        entrada_usuario = f"Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {row['estado_alimento']} {data_chave}."

        arguments = {
                    "numero_refeicao":None,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMAS",
                    "porcao_padrao":round(row["porcao_padrao"], 1),
                    "qtde_utilizada":qtde_utilizada,
                    "qtde_proteina":round(qtde_utilizada * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        qtde_utilizada * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(qtde_utilizada * row["qtde_gordura"], 1),
                    "qtde_alcool":round(qtde_utilizada * row["qtde_alcool"], 1),
                    "kcal":round(qtde_utilizada * row["kcal"], 1),
                },


        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Não é possível adicionar o alimento, Por favor informe o número da refeição.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem alimento

In [76]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(50).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        qtde_utilizada_porcao = random.randint(1, 5)
        qtde_utilizada_gramas = qtde_utilizada_porcao * row['porcao_padrao']
        qtde_utilizada = round(qtde_utilizada_gramas/row['porcao_padrao'],1)
        entrada_usuario = f"Adicionar {qtde_utilizada_gramas} gramas {row['estado_alimento']} na minha refeicao {numero_refeicao} de {data_chave}."

        arguments = {
                    "numero_refeicao":numero_refeicao,
                    "id_alimento":None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMAS",
                    "porcao_padrao":round(row["porcao_padrao"], 1),
                    "qtde_utilizada":qtde_utilizada,
                    "qtde_proteina":None,
                    "qtde_carboidrato":None,
                    "qtde_gordura":None,
                    "qtde_alcool":None,
                    "kcal":None,
                },

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Não é possível adicionar o alimento, Por favor informe o nome do alimento.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


### Sem Quantidade

In [77]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:

    for index, row in df.head(50).iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        qtde_utilizada_porcao = random.randint(1, 5)
        qtde_utilizada_gramas = qtde_utilizada_porcao * row['porcao_padrao']
        qtde_utilizada = round(qtde_utilizada_gramas/row['porcao_padrao'],1)
        entrada_usuario = f"Adicionar {row['nome_alimento']} {row['estado_alimento']} na minha refeicao {numero_refeicao} de {data_chave}."

        arguments = {
                    "numero_refeicao":numero_refeicao,
                    "id_alimento":None,
                    "id_prato":None,
                    "dt_dia":dates[data_chave],
                    "unidade_medida":"GRAMAS",
                    "porcao_padrao":None,
                    "qtde_utilizada":None,
                    "qtde_proteina":None,
                    "qtde_carboidrato":None,
                    "qtde_gordura":None,
                    "qtde_alcool":None,
                    "kcal":None,
                },


        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {
                    "role": "user",
                    "content": entrada_usuario
                },
                {
                    "role": "assistant",
                    "content": "Não é possível adicionar o alimento, Por favor informe a quantidade.",
                    "tool_calls": None,
                },
            ]
        }
        
        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + '\n')

print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.


# Assumir data Atual sem infor

In [78]:
with open(jsonl_file_path, 'a', encoding='utf-8') as jsonl_file:
    
    for index, row in df.iterrows():
        index_data = random.randint(0, 2)
        data_chave = datas_chaves[index_data]
        numero_refeicao = random.randint(1, 8)
        qtde_utilizada_porcao = random.randint(1, 5)
        qtde_utilizada_gramas = qtde_utilizada_porcao * row["porcao_padrao"]
        qtde_utilizada = round(qtde_utilizada_gramas / row["porcao_padrao"], 1)
        entrada_usuario = f"Adicionar {qtde_utilizada_gramas} gramas de {row['nome_alimento']} {row['estado_alimento']} na minha refeicao {numero_refeicao}."

        arguments = {
                    "numero_refeicao":numero_refeicao,
                    "id_alimento": None,
                    "id_prato":None,
                    "dt_dia":dates['hoje'],
                    "unidade_medida":"GRAMAS",
                    "porcao_padrao":round(row["porcao_padrao"], 1),
                    "qtde_utilizada":qtde_utilizada,
                    "qtde_proteina":round(qtde_utilizada * row["qtde_proteina"], 1),
                    "qtde_carboidrato":round(
                        qtde_utilizada * row["qtde_carboidrato"], 1
                    ),
                    "qtde_gordura":round(qtde_utilizada * row["qtde_gordura"], 1),
                    "qtde_alcool":round(qtde_utilizada * row["qtde_alcool"], 1),
                    "kcal":round(qtde_utilizada * row["kcal"], 1),
                },

        resposta_assistente = {
            "id": "call_V80O8ygGntZ7zyzexd2G7Jfd",
            "type": "function",
            "function": {
                "name": "add_consumo_alimento",
                "arguments": json.dumps(arguments)
            },
        }

        mensagem = {
            "messages": [
                {
                    "role": "system",
                    "content": content,
                },
                {"role": "user", "content": entrada_usuario},
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [resposta_assistente],
                },
            ]
        }

        jsonl_file.write(json.dumps(mensagem, ensure_ascii=False) + "\n")


print(f"Arquivo {jsonl_file_path} criado com sucesso.")

Arquivo output-backend.jsonl criado com sucesso.
